In [136]:
#read the distance and location files and create a panda dataframe for both
import os
import pandas as pd

project_root = os.path.abspath(os.getcwd())

data_dir = os.path.join(project_root, '..', 'data')

location_EV = pd.read_csv(os.path.join(data_dir, 'location_detail.csv'), sep=',')

distance_driven_EV = pd.read_csv(os.path.join(data_dir, 'distance_driven.csv'), sep=',')



In [137]:
print(location_EV.head())   
print(distance_driven_EV.head())

   Unnamed: 0     0   1   2   3   4   5   6   7   8  ... 1991 1992 1993 1994  \
0           0  0.00  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
1           1  0.25  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
2           2  0.50  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
3           3  0.75  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
4           4  1.00  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   

  1995 1996 1997 1998 1999 2000  
0   ho   ho   ho   ho   ho   ho  
1   ho   ho   ho   ho   ho   ho  
2   ho   ho   ho   ho   ho   ho  
3   ho   ho   ho   ho   ho   ho  
4   ho   ho   ho   ho   ho   ho  

[5 rows x 2002 columns]
   Unnamed: 0      0     1      2      3      4      5      6      7      8  \
0           0  155.0  54.0  203.0  149.0  138.0  116.0  161.0  180.0  311.0   

   ...  1990   1991   1992   1993   1994  1995  1996   1997   1998  1999  
0  ...  67.0  405.0  472.0  391.0  300.0  92.0  38.0  15

In [138]:
case_number = 7 
# Select the time column and the column "6" from both dataframes
time = location_EV.iloc[:, 1]  # Assuming the second column is time (first is Unnamed: 0)
location_col = location_EV.iloc[:, case_number]
distance_driven_case = int(distance_driven_EV.iloc[:, case_number])


# Combine into a single dataframe
ev_data = pd.DataFrame({
    'Time': time,
    'Location': location_col,
})



In [139]:
#count the amount of "dr" in the location column
dr_count = location_col.str.count('dr').sum()
print(f"Number of 'dr' in the location column: {dr_count}")


Number of 'dr' in the location column: 29


In [140]:
#add a new column called "distance driven" to the dataframe and fill it with 0 except for the rows where the location column constains "dr"
# there, distance_driven_case / dr_count
ev_data['Distance Driven'] = 0
ev_data.loc[location_col.str.contains('dr'), 'Distance Driven'] = distance_driven_case / dr_count



In [141]:
#add a new column called "EV_at_home" with 0 if car is not at home and 1 if car is at home
ev_data['EV_at_home'] = 0
ev_data.loc[location_col.str.contains('ho'), 'EV_at_home'] = 1

In [142]:
# add a new column calles "EV_charging_capacity" if 1 if car is at home, else 0 -> set this column equal to the "EV_at_home" column
ev_data['EV_charging_capacity'] = ev_data['EV_at_home']


# change time column -> 0.25 = 00:15:00

ev_data['Time'] = pd.to_timedelta(ev_data['Time'], unit='h').dt.components.apply(
    lambda x: f"{int(x.hours):02}:{int(x.minutes):02}:{int(x.seconds):02}", axis=1
)
print(ev_data)


         Time Location  Distance Driven  EV_at_home  EV_charging_capacity
0    00:00:00       ho              0.0           1                     1
1    00:15:00       ho              0.0           1                     1
2    00:30:00       ho              0.0           1                     1
3    00:45:00       ho              0.0           1                     1
4    01:00:00       ho              0.0           1                     1
..        ...      ...              ...         ...                   ...
667  22:45:00       ho              0.0           1                     1
668  23:00:00       ho              0.0           1                     1
669  23:15:00       ho              0.0           1                     1
670  23:30:00       ho              0.0           1                     1
671  23:45:00       ho              0.0           1                     1

[672 rows x 5 columns]


In [143]:
#out of this dataframe create a new dataframe with hourly data
# if car is at home in this hour, put 1 in "EV_at_home_hourly" else 0
# distance is added up for each hour
# the capacity is averaged for each hour 
#time to datetime, starting in august 2024
ev_data['Time'] = pd.to_datetime(ev_data['Time'], format='%H:%M:%S', errors='coerce')

# Create a list to hold each day's hourly dataframe (with datetime index)
ev_data_hourly_days = []
ev_data_hourly_days_winter = []

# There are 672 rows (15-min intervals for 7 days), so 24 hours per day, 4 intervals per hour
intervals_per_day = 24 * 4

# Start date for the week
start_date = pd.Timestamp('2023-07-31')
start_date_winter = pd.Timestamp('2023-11-27')

for day in range(7):
    day_slice = ev_data.iloc[day * intervals_per_day : (day + 1) * intervals_per_day].copy()
    # Set the correct date for this day
    current_date = start_date + pd.Timedelta(days=day)
    # Add the date to the time to get full datetime for each row
    day_slice['Datetime'] = current_date + pd.to_timedelta(day_slice['Time'].dt.hour, unit='h') \
                            + pd.to_timedelta(day_slice['Time'].dt.minute, unit='m') \
                            + pd.to_timedelta(day_slice['Time'].dt.second, unit='s')
    # Group by hour for this day using the Datetime column (floor to hour)
    day_slice['Hour'] = day_slice['Datetime'].dt.floor('H')
    day_hourly = day_slice.groupby('Hour').agg({
        'EV_at_home': 'max',
        'Distance Driven': 'sum',
        'EV_charging_capacity': 'mean'
    }).reset_index().rename(columns={
        'Hour': 'Datetime',
        'EV_at_home': 'EV_at_home_hourly'
    })
    # Add a column for the weekday
    day_hourly['Weekday'] = day
    # Save each day's hourly dataframe separately
    globals()[f'ev_data_hourly_day{day}'] = day_hourly
    ev_data_hourly_days.append(day_hourly)




# print one of the dataframes to check
print(ev_data_hourly_day0)

              Datetime  EV_at_home_hourly  Distance Driven  \
0  2023-07-31 00:00:00                  1         0.000000   
1  2023-07-31 01:00:00                  1         0.000000   
2  2023-07-31 02:00:00                  1         0.000000   
3  2023-07-31 03:00:00                  1         0.000000   
4  2023-07-31 04:00:00                  1         0.000000   
5  2023-07-31 05:00:00                  1         0.000000   
6  2023-07-31 06:00:00                  1         5.551724   
7  2023-07-31 07:00:00                  0         0.000000   
8  2023-07-31 08:00:00                  0         0.000000   
9  2023-07-31 09:00:00                  0         0.000000   
10 2023-07-31 10:00:00                  0         0.000000   
11 2023-07-31 11:00:00                  0         0.000000   
12 2023-07-31 12:00:00                  0         0.000000   
13 2023-07-31 13:00:00                  0         0.000000   
14 2023-07-31 14:00:00                  0         0.000000   
15 2023-

In [144]:
# add all the days together to one dataframe 
ev_data_hourly_1week = pd.concat([ev_data_hourly_day0,ev_data_hourly_day1,ev_data_hourly_day2,ev_data_hourly_day3,ev_data_hourly_day4,ev_data_hourly_day5,ev_data_hourly_day6], ignore_index=True)
ev_data_hourly_1week = ev_data_hourly_1week.sort_values(by='Datetime').reset_index(drop=True)


In [145]:
print(ev_data_hourly_1week)

# expand this dataframe to 4 weeks
ev_data_hourly_5weeks = pd.concat([ev_data_hourly_1week] * 5, ignore_index=True)
# Adjust the Datetime to reflect the correct weeks
ev_data_hourly_5weeks['Datetime'] += pd.to_timedelta(ev_data_hourly_5weeks.index // 168, unit='W')  # 168 hours in a week
print(ev_data_hourly_5weeks)
# Save the final dataframe to a CSV file
ev_data_hourly_5weeks.to_csv(os.path.join(data_dir, 'ev_data_hourly_5weeks_summer_2023.csv'), index=False)



               Datetime  EV_at_home_hourly  Distance Driven  \
0   2023-07-31 00:00:00                  1              0.0   
1   2023-07-31 01:00:00                  1              0.0   
2   2023-07-31 02:00:00                  1              0.0   
3   2023-07-31 03:00:00                  1              0.0   
4   2023-07-31 04:00:00                  1              0.0   
..                  ...                ...              ...   
163 2023-08-06 19:00:00                  1              0.0   
164 2023-08-06 20:00:00                  1              0.0   
165 2023-08-06 21:00:00                  1              0.0   
166 2023-08-06 22:00:00                  1              0.0   
167 2023-08-06 23:00:00                  1              0.0   

     EV_charging_capacity  Weekday  
0                     1.0        0  
1                     1.0        0  
2                     1.0        0  
3                     1.0        0  
4                     1.0        0  
..                   

In [146]:
for day in range(7):
    day_slice = ev_data.iloc[day * intervals_per_day : (day + 1) * intervals_per_day].copy()
    # Set the correct date for this day
    current_date = start_date_winter + pd.Timedelta(days=day)
    # Add the date to the time to get full datetime for each row
    day_slice['Datetime'] = current_date + pd.to_timedelta(day_slice['Time'].dt.hour, unit='h') \
                            + pd.to_timedelta(day_slice['Time'].dt.minute, unit='m') \
                            + pd.to_timedelta(day_slice['Time'].dt.second, unit='s')
    # Group by hour for this day using the Datetime column (floor to hour)
    day_slice['Hour'] = day_slice['Datetime'].dt.floor('H')
    day_hourly_winter = day_slice.groupby('Hour').agg({
        'EV_at_home': 'max',
        'Distance Driven': 'sum',
        'EV_charging_capacity': 'mean'
    }).reset_index().rename(columns={
        'Hour': 'Datetime',
        'EV_at_home': 'EV_at_home_hourly'
    })
    # Add a column for the weekday
    day_hourly_winter['Weekday'] = day
    # Save each day's hourly dataframe separately
    globals()[f'ev_data_hourly_day_winter{day}'] = day_hourly_winter
    ev_data_hourly_days_winter.append(day_hourly_winter)
    



ev_data_hourly_5weeks_winter = pd.concat([ev_data_hourly_1week_winter] * 5, ignore_index=True)
# Adjust the Datetime to reflect the correct weeks
ev_data_hourly_5weeks_winter['Datetime'] += pd.to_timedelta(ev_data_hourly_5weeks_winter.index // 168, unit='W')  # 168 hours in a week
# Save the final dataframe to a CSV file
ev_data_hourly_5weeks_winter.to_csv(os.path.join(data_dir, 'ev_data_hourly_5weeks_winter_2023.csv'), index=False)

In [147]:

ev_data_hourly_1week_winter = pd.concat([ev_data_hourly_day_winter0,ev_data_hourly_day_winter1,ev_data_hourly_day_winter2,ev_data_hourly_day_winter3,ev_data_hourly_day_winter4,ev_data_hourly_day_winter5,ev_data_hourly_day_winter6], ignore_index=True)
ev_data_hourly_1week_winter = ev_data_hourly_1week_winter.sort_values(by='Datetime').reset_index(drop=True)